In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians, floor
import glob

### Download list of GSOP stations

In [2]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv --directory-prefix data

--2018-06-19 11:16:00--  ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv
           => ‘data/isd-history.csv’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.101, 2610:20:8040:2::101
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.101|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/noaa ... done.
==> SIZE isd-history.csv ... 2922174
==> PASV ... done.    ==> RETR isd-history.csv ... done.
Length: 2922174 (2.8M) (unauthoritative)

isd-history.csv     100%[===================>]   2.79M  3.97MB/s    in 0.7s    

2018-06-19 11:16:01 (3.97 MB/s) - ‘data/isd-history.csv’ saved [2922174]



In [3]:
!wc -l data/isd-history.csv

29793 data/isd-history.csv


In [4]:
# load data, filter to US and Canada and stations with locations
stations = pd.read_csv('data/isd-history.csv')
stations = stations.loc[stations.CTRY.isin(['US', 'CA']), :]
stations = stations.loc[~((stations.LAT.fillna(0) == 0) & (stations.LON.fillna(0) == 0)), :]
stations.head(5)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
9641,423630,99999,MISSISSIPPI CANYON OIL PLATFORM,US,LA,NaN,28.160,-89.220,37.0,20130206,20160618
13036,619760,99999,SERGE-FROLOW (ILE TROMELIN),US,NaN,NaN,-15.883,54.517,13.0,19730101,20180617
13070,621010,99999,MOORED BUOY,US,NaN,NaN,50.600,-2.933,-999.0,20080721,20080721
13072,621110,99999,MOORED BUOY,US,NaN,NaN,58.900,-0.200,-999.0,20041118,20091201
13073,621130,99999,MOORED BUOY,US,NaN,NaN,58.400,0.300,-999.0,20040726,20040726


### Identify stations closest to MOBE

In [5]:
# location of mobe
mobe_lat, mobe_lon = (45.0150428, -93.456006)

# haversine distance between two points
def haversine(lat1, lon1, lat2=mobe_lat, lon2=mobe_lon):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [6]:
# distance of each station from MOBE
stations['distance'] = stations.apply(lambda row: haversine(row['LAT'], row['LON']), axis=1)
stations.sort_values(by=['distance'], ascending=True, inplace=True)
stations = stations.loc[(stations.BEGIN <= 19980000) & (stations.END >= 20170000), :]
stations.head(3)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,distance
20841,726580,14922,MINNEAPOLIS-ST PAUL INTERNATIONAL AP,US,MN,KMSP,44.883,-93.229,265.8,19450101,20180618,23.131246
20844,726584,14927,ST PAUL DWTWN HOLMAN FD AP,US,MN,KSTP,44.932,-93.056,213.4,19831103,20180618,32.802851
20792,726550,14926,ST CLOUD REGIONAL AIRPORT,US,MN,KSTC,45.543,-94.051,307.5,19730101,20180618,74.947609


### Download station/year files for closest stations

In [7]:
top3 = stations.apply(lambda row: '{}-{}'.format(row['USAF'], row['WBAN']), axis=1).head(3).tolist()
top3 = ' '.join(top3)

In [8]:
%%bash -s "{top3}"
for prefix in $@; do
    for year in {1998..2017}; do
        wget ftp://ftp.ncdc.noaa.gov/pub/data/gsod/$year/$prefix-$year.op.gz --quiet --directory-prefix data
        gunzip data/$prefix-$year.op.gz
    done
done

gzip: data/726584-14927-2000.op.gz: No such file or directory
gzip: data/726584-14927-2001.op.gz: No such file or directory
gzip: data/726584-14927-2002.op.gz: No such file or directory
gzip: data/726584-14927-2003.op.gz: No such file or directory


In [9]:
# load data
list_ = []
for file_ in glob.glob("data/*.op"):
    df = pd.read_fwf(file_)
    list_.append(df)
df = pd.concat(list_)

In [10]:
# cleanup
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
df['FRSHTT'] = df.FRSHTT.apply(lambda x: str(x).zfill(6))
df.head(5)

,STN---,WBAN,YEARMODA,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,726550,14926,19980101,22.3,17.7,1008.8,969.1,9.4,8.3,14.0,999.9,39.0*,8.1*,0.00G,999.9,100000
1,726550,14926,19980102,30.9,27.7,1005.0,966.3,9.2,5.8,11.1,999.9,39.0*,21.9*,0.00D,999.9,100000
2,726550,14926,19980103,16.4,11.3,1015.6,975.8,8.7,10.9,21.0,22.0,32.0*,7.0*,0.00C,999.9,100000
3,726550,14926,19980104,11.5,4.2,1027.2,986.3,9.9,5.9,10.1,999.9,24.1*,3.9*,0.00C,999.9,000000
4,726550,14926,19980105,24.3,22.0,1019.2,979.6,7.9,6.1,8.0,999.9,30.2*,21.0*,0.00H,999.9,111000


### Snow week analysis

In [11]:
# whether or not snow by day
df['snow'] = df.FRSHTT.apply(lambda x: int(x[2]))
df2 = df.groupby(['YEARMODA'], as_index=False)['snow'].max()
df2.sort_values(by=['YEARMODA'], ascending=True, inplace=True)
df2.reset_index(inplace=True, drop=True)
df2.head(10)

,YEARMODA,snow
0,19980101,0
1,19980102,0
2,19980103,0
3,19980104,0
4,19980105,1
5,19980106,1
6,19980107,1
7,19980108,0
8,19980109,1
9,19980110,0


In [12]:
# rolling sum
df2['snow_rolling'] = df2.rolling(window=7, min_periods=7).sum()['snow']
df2['snow_week'] = (df2.snow_rolling == 7).astype(int)
df2['year'] = df2.YEARMODA.apply(lambda x: floor(x/10000))
df3 = df2.groupby(['year'], as_index=False)['snow_week'].max()
df3

,year,snow_week
0,1998,1
1,1999,0
2,2000,1
3,2001,1
4,2002,1
5,2003,1
6,2004,1
7,2005,1
8,2006,0
9,2007,1


In [13]:
df3.snow_week.mean()

0.85